## EDA de Reporte Agregado TLC:

### 1. Importar librerías:

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
import numpy as np
warnings.filterwarnings('ignore')

### 2. Abrir documentos y extraerlos de la web:

In [2]:
# URL del archivo CSV
url = "https://www.nyc.gov/assets/tlc/downloads/csv/data_reports_monthly.csv"

# Leer el archivo CSV desde la URL
Aggregate_Report = pd.read_csv(url)

### 3. Renombrar datos y filtrar solo UberLyft, Amarillos y Verdes:

In [3]:
# Filtrar el DataFrame por 'License Class' y renombrar valores
Aggregate_Report = Aggregate_Report[Aggregate_Report['License Class'].isin(['FHV - High Volume', 'Green', 'Yellow'])]
Aggregate_Report['License Class'] = Aggregate_Report['License Class'].replace({'FHV - High Volume': 'UberLyft'})

### 4. Corregir formato de valores y disminuir tamaño:

In [4]:
# Reemplazar el símbolo "%" por una cadena vacía en la columna "Percent of Trips Paid with Credit Card"
Aggregate_Report['Percent of Trips Paid with Credit Card'] = Aggregate_Report['Percent of Trips Paid with Credit Card'].str.replace('%', '')

# Reemplazar los guiones "-" con ceros en las columnas que deseas convertir a float32
columns_to_replace = ['Trips Per Day', 'Farebox Per Day',
       'Unique Drivers', 'Unique Vehicles', 'Vehicles Per Day',
       'Avg Days Vehicles on Road', 'Avg Hours Per Day Per Vehicle',
       'Avg Days Drivers on Road', 'Avg Hours Per Day Per Driver',
       'Avg Minutes Per Trip', 'Percent of Trips Paid with Credit Card',
       'Trips Per Day Shared']

for column in columns_to_replace:
    if Aggregate_Report[column].dtype == 'object':  # Verificar si la columna es de tipo cadena
        Aggregate_Report[column] = Aggregate_Report[column].str.replace(',', '')

for column in columns_to_replace:
    Aggregate_Report[column] = Aggregate_Report[column].replace('-', '0')
    Aggregate_Report[column] = pd.to_numeric(Aggregate_Report[column], errors='coerce')  # Convertir a tipo float32, ignorando los errores de conversión

# Convertir los float64 a float32
Aggregate_Report[['Avg Days Vehicles on Road', 'Avg Hours Per Day Per Vehicle', 'Avg Days Drivers on Road', 'Avg Hours Per Day Per Driver', 'Avg Minutes Per Trip']] = Aggregate_Report[['Avg Days Vehicles on Road', 'Avg Hours Per Day Per Vehicle', 'Avg Days Drivers on Road', 'Avg Hours Per Day Per Driver', 'Avg Minutes Per Trip']].astype('float32')


Aggregate_Report[['Trips Per Day', 'Farebox Per Day', 'Unique Drivers', 'Unique Vehicles', 'Vehicles Per Day', 'Percent of Trips Paid with Credit Card', 'Trips Per Day Shared']] = Aggregate_Report[['Trips Per Day', 'Farebox Per Day', 'Unique Drivers', 'Unique Vehicles', 'Vehicles Per Day', 'Percent of Trips Paid with Credit Card', 'Trips Per Day Shared']].astype('float32')  # Convertir a float32


### 5. Filtrar datos desde el 2020 en adelante:

Month/Year en esta columna el año viene separado del mes con "-" por ejemplo 2024-01 para indicar enero de 2024. Se requiere separar mes y año y solo dejar los registros de los años 2020 en adelante:

In [5]:
# Separar la columna "Month/Year" en "Year" y "Month"
Aggregate_Report [['Year', 'Month']] = Aggregate_Report ['Month/Year'].str.split('-', expand=True)

# Convertir las columnas "Year" y "Month" a tipo numérico
Aggregate_Report ['Year'] = pd.to_numeric(Aggregate_Report ['Year'])
Aggregate_Report ['Month'] = pd.to_numeric(Aggregate_Report ['Month'])

# Filtrar para incluir solo los registros a partir de 2020
Aggregate_Report = Aggregate_Report [Aggregate_Report ['Year'] >= 2020]

# Convertir Year y Month a int16
Aggregate_Report[['Year', 'Month']] = Aggregate_Report[['Year', 'Month']].astype('int16')

# Filtrar para incluir solo los registros a partir de 2020
Aggregate_Report = Aggregate_Report[Aggregate_Report['Year'] >= 2020]


### 6. Ordenar columnas e eliminar las que no serán necesarias:

In [6]:
# Definir las dimensiones de agrupación y las variables de agregación
dimensiones = [ 'Year', 'Month', 'License Class']
variables_agregacion = ['Unique Drivers', 'Unique Vehicles', 'Vehicles Per Day',
       'Avg Days Vehicles on Road', 'Avg Hours Per Day Per Vehicle',
       'Avg Days Drivers on Road', 'Avg Hours Per Day Per Driver',
       'Avg Minutes Per Trip', 'Percent of Trips Paid with Credit Card',
       'Trips Per Day Shared']

# Reordenar las columnas para que primero estén las dimensiones y luego las variables de agregación
columnas_ordenadas = dimensiones + variables_agregacion

# Reordenar el DataFrame agrupado
Aggregate_Report = Aggregate_Report[columnas_ordenadas]

No se incluirán las columnas 'Trips Per Day', 'Farebox Per Day' por ser campos que se calculan en el dataset de viajes y en el caso de Farebox no hay datos de Uber en este datwa set.